In [1]:
# Load data - run prep_exp.ipynb first
import pickle

pickle_in = open("temp.pkl","rb")
data = pickle.load(pickle_in)

# # If it's not there, do the following:
# import sys
# import os
# import numpy as np
# from str2bool import str2bool
# from rpy2.robjects.packages import STAP
# import rpy2.robjects as robjects
# from rpy2.robjects import pandas2ri, numpy2ri
# from rpy2.robjects.lib.dplyr import DataFrame
# from rpy2.robjects.packages import importr

# pandas2ri.activate()
# numpy2ri.activate()

# # Some temporary arguments for testing
# include_lab = str2bool("T")  # Include lab features?
# include_ethdon = str2bool("T")  # Include ethnicity + donor details?
# lag = int("1")  # Number of lag variables
# eq_train_ratio = str2bool("T")  # Train on equal case:control ratio?
# __file__ = '/h/angeliney/projects/SRTR/prep_exp.py'
# visit_type = "first"
# output = "temp"
# post2000 = True

# # Get features based on these inputs
# with open(os.path.join(os.path.dirname(__file__), 'features.R'), 'r') as f:
#     string = f.read()
# features_file = STAP(string, "features")
# features_to_use = features_file.features.rx2("clin")
# if include_lab:
#     features_to_use = features_to_use + features_file.features.rx2("lab")

# if include_ethdon:
#     features_to_use = features_to_use + features_file.features.rx2("eth") + features_file.features.rx2("don")

# timedep_cols = np.intersect1d(features_to_use, features_file.timedep_features)
# cov_cols = np.setdiff1d(features_to_use, timedep_cols)

# if eq_train_ratio:
#     eq_cases_train_cols = np.array(["TRR_ID", "is_diab"])
# else:
#     eq_cases_train_cols = np.array()
    
# # Read RDS files (load data table)
# readRDS = robjects.r['readRDS']
# tx_li_study = readRDS(os.path.join(os.path.dirname(__file__), 'tx_li_formatted.rds'))
# txf_li_study = readRDS(os.path.join(os.path.dirname(__file__), 'txf_li_formatted.rds'))

# # Merge them
# with open(os.path.join(os.path.dirname(__file__), 'functions.R'), 'r') as f:
#     string = f.read()
# functions = STAP(string, "functions")

# merged = functions.combine_tx_txf(tx_li_study, txf_li_study, np.setdiff1d(cov_cols, "age"), timedep_cols, lag)

# df = pandas2ri.ri2py_dataframe(DataFrame(merged).filter('time_next_followup > time_since_transplant'))

# #Prep data for model training
# cols = np.concatenate((timedep_cols, cov_cols))
# if lag > 0:
#     for l in range(1,  lag + 1):
#         cols = np.append(cols, list(map(lambda x: '{}_{}'.format(x, l), timedep_cols)))

# subset_cols = np.concatenate((['transplant_year', 'TRR_ID', 'age'], cols, ['is_diab', 'time_since_transplant',
#                                                                    'time_next_followup', 'time_to_diab',
#                                                                    'diab_time_since_tx', 'diab_in_1_year',
#                                                                    'diab_now']))
# df = df.dropna(subset=subset_cols)
# df_test = df[(df.transplant_year.astype(int) >= 2011) & (df.time_to_diab >= 0)]
# df_nontest = df[(df.transplant_year.astype(int) < 2011) & (df.time_to_diab >= 0)]

# num_folds = 5
# nontest_y = df_nontest.drop_duplicates(subset=['TRR_ID', 'is_diab']).is_diab
# caret = importr('caret')
# folds = caret.createFolds(nontest_y, num_folds, False)

# data = {'test': df_test, 'train': df_nontest, 'cols': cols, 'eq_cases_train_cols': eq_cases_train_cols,
#             'folds': folds}

# import pickle

# # Save data in case kernel got restarted
# pickle.dump(data, open("temp.pkl", "wb")) 

In [2]:
# Try MERF
import numpy as np
import pandas
import sys
import pickle
from merf import MERF
from sklearn.metrics import roc_auc_score
from str2bool import str2bool
from functions import prep_exp, filter_train_by_visit

cols = np.append(data['cols'], 'time_since_transplant')
nontest_ids = data['train'].drop_duplicates(subset=['TRR_ID', 'is_diab']).TRR_ID


/h/angeliney/miniconda2/lib/python2.7/site-packages/sklearn/utils/__init__.py:12: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/h/angeliney/miniconda2/lib/python2.7/site-packages/sklearn/utils/extmath.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/h/angeliney/miniconda2/lib/python2.7/site-packages/sklearn/utils/extmath.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms
/h/angeliney/miniconda2/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/h/angeliney/miniconda2/lib/python2.7/site-packages/skl

In [ ]:
result = pandas.DataFrame()
for i in range(num_folds+1):
    if i > 0:
        train_ids = nontest_ids[np.array(data['folds']) != i]
        val_ids = np.setdiff1d(nontest_ids, train_ids)
        test = data['train'][data['train']['TRR_ID'].isin(val_ids)]
    else:
        train_ids = nontest_ids
        test = data['test']

    train = filter_train_by_visit(visit_type, data['train'][data['train'].TRR_ID.isin(train_ids)])

    merf = MERF(n_estimators=100, gll_early_stop_threshold=0.001, max_iterations=2)
    merf.fit(train[cols], pandas.DataFrame(np.ones((train.shape[0], 1))), train.TRR_ID, train.diab_in_1_year)
    train_y_hat = merf.predict(train[cols], pandas.DataFrame(np.ones((train.shape[0], 1))), train.TRR_ID)
    test_y_hat = merf.predict(test[cols], pandas.DataFrame(np.ones((test.shape[0], 1))), test.TRR_ID)

    train_auroc = roc_auc_score(train.diab_in_1_year, train_y_hat)
    test_auroc = roc_auc_score(test.diab_in_1_year, test_y_hat)

    if i == 0:
        perf = {'model': merf, 'train_auroc': train_auroc, 'test_auroc': test_auroc,
                'train_nrow': train.shape[0], 'test_nrow': test.shape[0]}
        pickle.dump(perf, open('{}_perf.pkl'.format(output), 'wb'))

    print('{},{},{},{},{}'.format(i, train_auroc, test_auroc, train.shape[0], test.shape[0]))
    result.append(pandas.DataFrame({'train_auroc': train_auroc, 'test_auroc': test_auroc,
                                    'train_nrow': train.shape[0], 'test_nrow': test.shape[0]}, index=[0]))

In [4]:
i=1
if i > 0:
    train_ids = nontest_ids[np.array(data['folds']) != i]
    val_ids = np.setdiff1d(nontest_ids, train_ids)
    test = data['train'][data['train']['TRR_ID'].isin(val_ids)]
else:
    train_ids = nontest_ids
    test = data['test']

train = filter_train_by_visit("all", data['train'][data['train'].TRR_ID.isin(train_ids)])

merf = MERF(n_estimators=100, gll_early_stop_threshold=0.001, max_iterations=2)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas
import sys
import pickle

i=1
if i > 0:
    train_ids = nontest_ids[np.array(data['folds']) != i]
    val_ids = np.setdiff1d(nontest_ids, train_ids)
    test = data['train'][data['train']['TRR_ID'].isin(val_ids)]
else:
    train_ids = nontest_ids
    test = data['test']

train = filter_train_by_visit("all", data['train'][data['train'].TRR_ID.isin(train_ids)])
model = RandomForestClassifier(n_estimators=100, max_depth=10)
model.fit(train[cols], train.diab_in_1_year)
train_y_hat = model.predict(train[cols])
test_y_hat = model.predict(test[cols])

train_auroc = roc_auc_score(train.diab_in_1_year, train_y_hat)
test_auroc = roc_auc_score(test.diab_in_1_year, test_y_hat)